In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_03_17=pd.read_csv('/kaggle/input/singapore-waste-management/2003_2017_waste.csv')
df_18_20=pd.read_csv('/kaggle/input/singapore-waste-management/2018_2020_waste.csv')
df_we=pd.read_csv('/kaggle/input/singapore-waste-management/waste_energy_stat.csv')

# 1. Data Cleaning

Theare are some difference in colums' names between df_03_17 and df_18_20. So I tried to match both columns' names. And I found some outliers in data df_18_20, so I changed them to the numbers which seem to be appropriate.

Next, I made one date frame which merged df_03_17 with df_18_20.

And the values of 'waste_type' have lots of different names which are actually same type. So I tried to match them into same types. 

1)Data Cleaning

In [ ]:
df_03_17.info()

In [ ]:
df_03_17.head()

In [ ]:
df_18_20.info()

In [ ]:
df_18_20.head()

In [ ]:
df_we.info()

In [ ]:
df_we

In [ ]:
df_18_20 = df_18_20.rename(columns={'Waste Type':'waste_type', "Total Generated ('000 tonnes)":'total_waste_generated_tonne',
                              "Total Recycled ('000 tonnes)":'total_waste_recycled_tonne','Year':'year'})

In [ ]:
df_18_20.head()

In [ ]:
df_18_20.iloc[19,1]=744
df_18_20.iloc[1,2]=1260

2)Data Merging

In [ ]:
df_18_20['total_waste_generated_tonne']=df_18_20['total_waste_generated_tonne']*1000
df_18_20['total_waste_recycled_tonne']=df_18_20['total_waste_recycled_tonne']*1000
df_18_20['recycling_rate']=df_18_20['total_waste_recycled_tonne']/df_18_20['total_waste_generated_tonne']

In [ ]:
df_18_20['recycling_rate']=df_18_20['recycling_rate'].round(2)

In [ ]:
df_18_20.head()

In [ ]:
df_18_20 = df_18_20.reindex(columns=['waste_type','total_waste_recycled_tonne','total_waste_generated_tonne','recycling_rate','year'])

In [ ]:
df_18_20.head()

In [ ]:
df_03_17=df_03_17.drop('waste_disposed_of_tonne',axis=1)

In [ ]:
df_03_17.head()

In [ ]:
df=pd.concat([df_03_17, df_18_20])

3)The value cleaning of column 'waste_type'

In [ ]:
df['waste_type'].unique()

In [ ]:
df['waste_type']=df['waste_type'].str.lower()

In [ ]:
df['waste_type'].value_counts()

In [ ]:
df=df.replace('plastic','plastics') 
df=df.replace('overall','total')
df=df.replace('horticultural','horticultural waste')
df=df.replace('ferrous metal','ferrous metals')
df=df.replace('non-ferrous metal','non-ferrous metals')
df=df.replace(['construction& demolition','construction & demolition','c&d'],'construction debris')
df=df.replace(['others (stones, ceramic, rubber, ect)','others (stones, ceramics, etc.)','others',
               'others (stones, ceramic, rubber, etc.)','others (stones, ceramics & rubber etc.)'],'others (stones, ceramics & rubber etc)')
df=df.replace('food waste','food')
df=df.replace('wood','wood/timber')
df=df.replace(['ash & sludge','sludge'],'ash and sludge')

In [ ]:
df['waste_type'].value_counts()

I got one data frame.

# 2. Visualization

First, I tried to understand time series trend of 'total waste generated', 'total recycle generated' and 'recycling rate'and the portion of 'total_waste_generated_tonne' by 'waste_type'.

Next, I tried to understand the amount and time series trend of 'energy_saving_amount' and 'oil_saving_amount' by 'waste_type'.

Lastly, I also tried to understand time series trend of 'waste generated', 'recycle generated' and 'recycling rate' by some 'waste_type' to get some insight.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime

In [ ]:
df_total=df[df['waste_type']=='total']

In [ ]:
df_total= df_total.sort_values('year')

In [ ]:
df_total.head()

In [ ]:
df_total.dtypes

1)Time series trend of 'total waste generated', 'total recycle generated' and 'recycling rate'

In [ ]:
fig = go.Figure()

fig= make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(
    go.Bar(
        x=df_total['year'],
        y=df_total['total_waste_generated_tonne'],
        name='Waste Generated',
    )
)
fig.add_trace(
    go.Bar(
        x=df_total['year'],
        y=df_total['total_waste_recycled_tonne'],
        name='Waste Recycled',
    )
)

fig.add_trace(
    go.Line(
        x=df_total['year'],
        y=df_total['recycling_rate'],
        name='Recycling Rate',
    ),
    secondary_y=True
)

fig.show()

2)The portion of 'total_waste_generated_tonne' by 'waste_type'

In [ ]:
df1=df[df['waste_type'] != 'total']
df1.groupby('waste_type')['total_waste_generated_tonne'].sum().plot.pie(figsize=(8,8),autopct='%1.1f%%')

3)Calcurate the amount of 'energy_saving_amount' and 'oil_saving_amount'

In [ ]:
df_energy_save=df[df['waste_type'].isin(['plastics','glass','ferrous metals','non-ferrous metals','paper/cardboard'])]

In [ ]:
df_energy_save.head()

In [ ]:
def energy_save(ex):
    if ex=='paper/cardboard':
        return 4100
    elif ex=='plastics':
        return 5774
    elif ex=='ferrous metals':
        return 642
    elif ex=='non-ferrous metals':
        return 14000
    else:
        return 42

In [ ]:
def oil_saving(ex):
    if ex=='paper/cardboard':
        return 11*159
    elif ex=='plastics':
        return 16*159
    elif ex=='ferrous metals':
        return 1.8*159
    elif ex=='non-ferrous metals':
        return 40*159
    else:
        return 0.12*159

In [ ]:
df_energy_save['energy_saving']=df_energy_save['waste_type'].apply(energy_save)
df_energy_save['oil_saving']=df_energy_save['waste_type'].apply(oil_saving)
df_energy_save['energy_saving_amount']=df_energy_save['total_waste_recycled_tonne']*df_energy_save['energy_saving']
df_energy_save['oil_saving_amount']=df_energy_save['total_waste_recycled_tonne']*df_energy_save['oil_saving']

In [ ]:
df_energy_save.head()

4)The time series trend of 'energy_saving_amount' and 'oil_saving_amount' by 'waste_type'

In [ ]:
fig = px.bar(df_energy_save, x="year", y="energy_saving_amount", color="waste_type", title="energy_saving_amount")
fig.show()

In [ ]:
fig = px.bar(df_energy_save, x="year", y="oil_saving_amount", color="waste_type", title="oil_saving_amount")
fig.show()

Paper/Cardboard and non-ferrous metals are larger impact to energy saving because their amount of energy saved in kilowatt hour (kWh) and their amount of crude oil (barrels) by recycling 1 metric tonne (1000 kilogram) per waste type are large.

5)The time series trend of 'waste generated', 'recycle generated' and 'recycling rate' by some 'waste_types' 

In [ ]:
df_metal=df[df['waste_type']=='ferrous metals'].sort_values('year')
df_const=df[df['waste_type']=='construction debris'].sort_values('year')
df_food=df[df['waste_type']=='food'].sort_values('year')
df_paper=df[df['waste_type']=='paper/cardboard'].sort_values('year')
df_plastic=df[df['waste_type']=='plastics'].sort_values('year')
df_nometal=df[df['waste_type']=='non-ferrous metals'].sort_values('year')
df_glass=df[df['waste_type']=='glass'].sort_values('year')

In [ ]:
fig = go.Figure()

fig= make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(
    go.Bar(
        x=df_metal['year'],
        y=df_metal['total_waste_generated_tonne'],
        name='Metal Waste Generated',
    )
)
fig.add_trace(
    go.Bar(
        x=df_metal['year'],
        y=df_metal['total_waste_recycled_tonne'],
        name='Metal Waste Recycled',
    )
)

fig.add_trace(
    go.Line(
        x=df_metal['year'],
        y=df_metal['recycling_rate'],
        name='Metal Recycling Rate',
    ),
    secondary_y=True
)

fig.show()

In [ ]:
fig = go.Figure()

fig= make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(
    go.Bar(
        x=df_const['year'],
        y=df_const['total_waste_generated_tonne'],
        name='Construction Waste Generated',
    )
)
fig.add_trace(
    go.Bar(
        x=df_const['year'],
        y=df_const['total_waste_recycled_tonne'],
        name='Construction Waste Recycled',
    )
)

fig.add_trace(
    go.Line(
        x=df_const['year'],
        y=df_const['recycling_rate'],
        name='Construction Recycling Rate',
    ),
    secondary_y=True
)

fig.show()

In [ ]:
fig = go.Figure()

fig= make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(
    go.Bar(
        x=df_food['year'],
        y=df_food['total_waste_generated_tonne'],
        name='Food Waste Generated',
    )
)
fig.add_trace(
    go.Bar(
        x=df_food['year'],
        y=df_food['total_waste_recycled_tonne'],
        name='Food Waste Recycled',
    )
)

fig.add_trace(
    go.Line(
        x=df_food['year'],
        y=df_food['recycling_rate'],
        name='Food Recycling Rate',
    ),
    secondary_y=True
)

fig.show()

In [ ]:
fig = go.Figure()

fig= make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(
    go.Bar(
        x=df_paper['year'],
        y=df_paper['total_waste_generated_tonne'],
        name='Paper Waste Generated',
    )
)
fig.add_trace(
    go.Bar(
        x=df_paper['year'],
        y=df_paper['total_waste_recycled_tonne'],
        name='Paper Waste Recycled',
    )
)

fig.add_trace(
    go.Line(
        x=df_paper['year'],
        y=df_paper['recycling_rate'],
        name='Paper Recycling Rate',
    ),
    secondary_y=True
)

fig.show()

In [ ]:
fig = go.Figure()

fig= make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(
    go.Bar(
        x=df_plastic['year'],
        y=df_plastic['total_waste_generated_tonne'],
        name='Plastic Waste Generated',
    )
)
fig.add_trace(
    go.Bar(
        x=df_plastic['year'],
        y=df_plastic['total_waste_recycled_tonne'],
        name='Plastic Waste Recycled',
    )
)

fig.add_trace(
    go.Line(
        x=df_plastic['year'],
        y=df_plastic['recycling_rate'],
        name='Plastic Recycling Rate',
    ),
    secondary_y=True
)

fig.show()

In [ ]:
fig = go.Figure()

fig= make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(
    go.Bar(
        x=df_nometal['year'],
        y=df_nometal['total_waste_generated_tonne'],
        name='Nonmetal Waste Generated',
    )
)
fig.add_trace(
    go.Bar(
        x=df_nometal['year'],
        y=df_nometal['total_waste_recycled_tonne'],
        name='Nonmetal Waste Recycled',
    )
)

fig.add_trace(
    go.Line(
        x=df_nometal['year'],
        y=df_nometal['recycling_rate'],
        name='Nonmetal Recycling Rate',
    ),
    secondary_y=True
)

fig.show()

In [ ]:
fig = go.Figure()

fig= make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(
    go.Bar(
        x=df_glass['year'],
        y=df_glass['total_waste_generated_tonne'],
        name='Glass Waste Generated',
    )
)
fig.add_trace(
    go.Bar(
        x=df_glass['year'],
        y=df_glass['total_waste_recycled_tonne'],
        name='Glass Waste Recycled',
    )
)

fig.add_trace(
    go.Line(
        x=df_glass['year'],
        y=df_glass['recycling_rate'],
        name='Glass Recycling Rate',
    ),
    secondary_y=True
)

fig.show()

# 3.The insight

1)Paper/Cardboard and non-ferrous metals are larger impact to energy saving because their amount of energy saved in kilowatt hour (kWh) and their amount of crude oil (barrels) by recycling 1 metric tonne (1000 kilogram) per waste type are large. 

2)Plastic has also the largest amount of energy saved in kilowatt hour (kWh) and larger amount of crude oil (barrels) by recycling 1 metric tonne (1000 kilogram) per waste type. But its recycle rate is very low, so the impact to energy save is quiet low. One of the important action which we should consider is how we can improve the tech to raise the recycle rate of plastics.

3)Other problem is food waste globally. In Singapore, it seems same as other advanced countries. This time we do not have any data about their amount of energy saved in kilowatt hour (kWh) and their amount of crude oil (barrels) by recycling 1 metric tonne (1000 kilogram) per waste type. But if we can improve its recycle rate and understand data about energy save and crude oil by recycling, we may be improve more totally.